# Detecting stops

<img align="right" src="https://anitagraser.github.io/movingpandas/assets/img/movingpandas.png">

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/anitagraser/movingpandas-examples/main?filepath=1-tutorials/8-detecting-stops.ipynb)
[![IPYNB](https://img.shields.io/badge/view-ipynb-hotpink)](https://github.com/anitagraser/movingpandas-examples/blob/main/1-tutorials/8-detecting-stops.ipynb)
[![HTML](https://img.shields.io/badge/view-html-green)](https://anitagraser.github.io/movingpandas-website/1-tutorials/8-detecting-stops.html)

There are no definitive answers when it comes to detecting / extracting stops from movement trajectories. Due to tracking inaccuracies, movement speed rarely goes to true zero. GPS tracks, for example, tend to keep moving around the object's stop location.

Suitable stop definitions are also highly application dependent. For example, an application may be interested in analyzing trip purposes. To do so, analysts would be interested in stops that are longer than, for example, 5 minutes and may try to infer the purpose of the stop from the stop location and time. Shorter stops, such as delays at traffic lights, however would not be relevant for this appication.

In the MovingPandas TrajectoryStopDetector implementation, a stop is detected if the movement stays within an area of specified size for at least the specified duration.

In [ ]:
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame, read_file
from shapely.geometry import Point, LineString, Polygon
from datetime import datetime, timedelta
import movingpandas as mpd

import warnings
warnings.filterwarnings('ignore')

mpd.show_versions()

In [ ]:
FSIZE = 350

In [ ]:
gdf = read_file('../data/geolife_small.gpkg')
traj_collection = mpd.TrajectoryCollection(gdf, 'trajectory_id', t='t')

## Stop detection with a single Trajectory

In [ ]:
my_traj = traj_collection.trajectories[0]
my_traj

In [ ]:
traj_plot = my_traj.hvplot(title='Trajectory {}'.format(my_traj.id), line_width=7.0, tiles='CartoLight', color='slategray', frame_width=FSIZE, frame_height=FSIZE) 
traj_plot

In [ ]:
detector = mpd.TrajectoryStopDetector(my_traj)

### Stop duration

In [ ]:
%%time
stop_time_ranges = detector.get_stop_time_ranges(min_duration=timedelta(seconds=60), max_diameter=100)

In [ ]:
for x in stop_time_ranges: 
    print(x)

### Stop points

In [ ]:
help(mpd.TrajectoryStopDetector.get_stop_points)

In [ ]:
%%time
stop_points = detector.get_stop_points(min_duration=timedelta(seconds=60), max_diameter=100)

In [ ]:
stop_points

In [ ]:
stop_point_plot = traj_plot * stop_points.hvplot(geo=True, size='duration_s', color='deeppink')
stop_point_plot

### Stop segments

In [ ]:
help(mpd.TrajectoryStopDetector.get_stop_segments)

In [ ]:
%%time
stops = detector.get_stop_segments(min_duration=timedelta(seconds=60), max_diameter=100)

In [ ]:
stops

In [ ]:
stop_segment_plot = stop_point_plot * stops.hvplot( size=200, line_width=7.0, tiles=None, color='orange') 
stop_segment_plot

### Split at stops

In [ ]:
help(mpd.StopSplitter)

In [ ]:
%%time
split = mpd.StopSplitter(my_traj).split(min_duration=timedelta(seconds=60), max_diameter=100)

In [ ]:
split

In [ ]:
split.to_traj_gdf()

In [ ]:
stop_segment_plot + split.hvplot(title='Trajectory {} split at stops'.format(my_traj.id), line_width=7.0, tiles='CartoLight', frame_width=FSIZE, frame_height=FSIZE)

## Stop Detection for TrajectoryCollections

The process is the same as for individual trajectories.

In [ ]:
%%time
detector = mpd.TrajectoryStopDetector(traj_collection)
stop_points = detector.get_stop_points(min_duration=timedelta(seconds=120), max_diameter=100)
len(stop_points)

In [ ]:
ax = traj_collection.plot(figsize=(7,7))
stop_points.plot(ax=ax, color='red',  )